In [43]:
from riotwatcher import RiotWatcher
from riotwatcher import LoLException, error_500
import time
import os
import json
import pandas as pd
w = RiotWatcher('b6a1a049-887c-4b5e-bc1f-404b5a43e48d',default_region='LAS')

In [3]:
# Check if game was already fetched
def check_file_exists(path):
    try:
        text_file = open(path,'r')
        # print 'Repeated Game'
        text_file.close()
        return False
    except IOError:
        return True 

In [4]:
# Search for ranked matches based on seed players
def search_games(seed_players, checked_players):
    fellow_players = []
    for player in seed_players:
        if player not in checked_players:
            # Get recent games.
            # print 'Checking Player %s' % str(player)
            try:
                time.sleep(1.2)
                recent_games = w.get_recent_games(player)
                for game in recent_games['games']:
                    # Check if game is ranked
                    if game['subType'] == 'RANKED_SOLO_5x5' or game['subType'] == 'RANKED_PREMADE_5x5':
                        path = str(os.getcwd()) + "/Games/" + str(game['gameId']) + ".json"
                        # Check if game was saved before
                        if check_file_exists(path):
                            time.sleep(1.2)
                            # Get game data and save json
                            try:
                                game_data = w.get_match(game['gameId'], include_timeline=True) 
                                with open(path, "w") as text_file:
                                    json.dump(game_data,text_file)
                                text_file.close() 
                            except LoLException as error:
                                if error == error_500:
                                    print 'Error 500'
                    # Get fellow players
                    try:
                        for player in game['fellowPlayers']:
                            fellow_players.append(player['summonerId'])
                    except KeyError:
                        print 'No fellow Players'
            except LoLException as error:
                if error == error_500:
                    print 'Error 500'
        else:
            print 'Repeated Player'
    return fellow_players

In [5]:
seed_players = []
searched_players = []
seed_players.append(w.get_summoner(name='flsqm')['id'])
seed_players.append(w.get_summoner(name='jiezou11')['id'])
seed_players.append(w.get_summoner(name='Ssvage')['id'])
seed_players.append(w.get_summoner(name='whitebone45')['id'])
seed_players.append(w.get_summoner(name='Cafez')['id'])
time.sleep(12)
fellow_players = search_games(seed_players, searched_players)
searched_players = searched_players + seed_players
fellow_players2 = search_games(fellow_players,searched_players)
searched_players = searched_players + fellow_players
fellow_players3 = search_games(fellow_players2,searched_players)
searched_players = searched_players + fellow_players2
fellow_players4 = search_games(fellow_players3,searched_players)
searched_players = searched_players + fellow_players3

ConnectionError: HTTPSConnectionPool(host='natr.api.pvp.net', port=443): Max retries exceeded with url: /api/lol/NATR/v1.4/summoner/by-name/flsqm?api_key=b6a1a049-887c-4b5e-bc1f-404b5a43e48d (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x0000000005E9BEF0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))